# 航拍仙人掌预测 

In [53]:
{
    print("Author: EugeneLiu")
}

Author: EugeneLiu


{None}

## 导入需要的 

In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
print(os.listdir("./"))

['test', 'train', 'train.csv', '仙人掌识别.ipynb', 'processedPictures', '.ipynb_checkpoints', 'sample_submission.csv', 'tfrecord', '未命名.ipynb']


In [4]:
import tensorflow as tf
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

## 文件路径设置和train文件的读取 

In [5]:
train_dir = "./train"
test_dir = "./test"

train = pd.read_csv("./train.csv")

In [6]:
train.head(5)

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [7]:
type(train.has_cactus[0])

numpy.int64

In [8]:
train.shape

(17500, 2)

In [9]:
train.has_cactus.value_counts()

1    13136
0     4364
Name: has_cactus, dtype: int64

In [10]:
# 预备对象待时机写入到TFRecord文件中
tfrecord_filename = ("train_data-000")
writer = tf.python_io.TFRecordWriter("./tfrecord/"+tfrecord_filename)

In [11]:
# 将数据写入到TFRecord中
def create_record():
    for row_number,row in train.iterrows():
#         print(row["id"]+"  "+str(row["has_cactus"]))
        imgpath =train_dir + "/" + row["id"]
#         print(imgpath)
        img = Image.open(imgpath,"r")
        img_raw = img.tobytes()
#         print(kind+"   "+img_raw)
        examples = tf.train.Example(
            features=tf.train.Features(
                feature={
                    "label" : tf.train.Feature(int64_list=tf.train.Int64List(value=[row["has_cactus"]])),
                    "img_raw" : tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
                }
            )
        )
#         print(img_raw)
        writer.write(examples.SerializeToString())
    writer.close()
        
create_record()        
#        print(filepath)

In [12]:
def load_data_and_decode_data():
    reader = tf.TFRecordReader()
    _,serialized_example = reader.read(tf.train.string_input_producer(["./tfrecord/train_data-000"]))
    features = tf.parse_single_example(serialized_example,
                                      features={
                                              "label":tf.FixedLenFeature([],tf.int64),
                                              "img_raw":tf.FixedLenFeature([],tf.string)
                                      })
    label = features["label"]
    img = features["img_raw"]
#     print(type(label))
#     print(type(img))
    img = tf.decode_raw(img,tf.uint8)
    img = tf.reshape(img,[32,32,3])
    img = tf.cast(img,tf.float32) * (1. / 255)
    label = tf.cast(label,tf.float32)
    return img,label

In [13]:
batch = load_data_and_decode_data()

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cas

In [14]:
init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())

In [15]:
train.shape[0]

17500

In [17]:
with tf.Session() as sess:
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord = coord)
    for i in range(train.shape[0]):
        example,label = sess.run(batch)
#         print(example)
#         print(label)
        img = Image.fromarray(example,'RGB')
        img.save("./processedPictures/"+str(i)+"_"+ str(int(label)) +".jpg")
    coord.request_stop()
    coord.join(threads)
    sess.close()

Instructions for updating:
To construct input pipelines, use the `tf.data` module.


## 分验证集和测试集

In [18]:
from sklearn.model_selection import train_test_split

In [26]:
data = []
label = []
for imgPath in os.listdir("./processedPictures"):
    data.append("./processedPictures/"+imgPath)
    label.append(imgPath.split("_")[1].split(".")[0])
#     print(imgPath.split("_")[1].split(".")[0])

In [35]:
data_train,data_validation,label_train,label_validation = train_test_split(data,label,test_size=0.3,random_state=0)

In [40]:
data_train[:10]

['./processedPictures/5433_1.jpg',
 './processedPictures/12066_1.jpg',
 './processedPictures/16971_1.jpg',
 './processedPictures/1397_1.jpg',
 './processedPictures/280_0.jpg',
 './processedPictures/17068_1.jpg',
 './processedPictures/749_1.jpg',
 './processedPictures/10100_0.jpg',
 './processedPictures/15622_1.jpg',
 './processedPictures/1859_1.jpg']

In [41]:
data_validation[:10]

['./processedPictures/1718_1.jpg',
 './processedPictures/6638_1.jpg',
 './processedPictures/2484_1.jpg',
 './processedPictures/1462_1.jpg',
 './processedPictures/1651_1.jpg',
 './processedPictures/14316_1.jpg',
 './processedPictures/13448_1.jpg',
 './processedPictures/14207_1.jpg',
 './processedPictures/11744_1.jpg',
 './processedPictures/6639_1.jpg']

In [43]:
label_train[:10]

['1', '1', '1', '1', '0', '1', '1', '0', '1', '1']

In [44]:
label_validation[:10]

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']

In [52]:
len(data_train)

12250

## 使用tensorflow搭建卷积神经网络 

In [55]:
32 * 32 * 3

3072

In [76]:
INPUT_NODE = 32 * 32 * 3 #3072
OUTPUT_NODE = 2

IMAGE_SIZE =32
NUM_CHANNELS = 3#RGB为三个维度,图层数目为1
NUM_LABELS = 1

CONV1_DEEP = 6
CONV1_SIZE = 5
CONV1_STEP = 1

CONV2_DEEP = 16
CONV2_SIZE = 5
CONV2_STEP = 1

FC_SIZE = 120 #自己设置第五层的全连接层的节点数目
FC2_SIZE = 84 #自己设置第六层的全连接层的节点数目
FC3_SIZE = 2

In [77]:
def forward_propagation(input_tensor,is_train,regularizer):#一次处理的图片,批处理大小,可预测种类数
    with tf.variable_scope("conv1"):#名字为conv1,看打印日志的时候需要看这个
        conv1_weights = tf.Variable("weight",[CONV1_SIZE,CONV1_SIZE,NUM_CHANNELS,CONV1_DEEP],
                              initializer = tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases = tf.get_variable("bias",[CONV1_DEEP],initializer = tf.constant_initializer(0.0))
        conv1 = tf.nn.conv2d(input_tensor,conv1_weights,strides=[1,1,1,1],padding='VALID')
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1,conv1_biases))
    with tf.name_scope("pool1"):
        pool1 = tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],paddind='VALID')
    with tf.variable_scope("conv2"):
        #定义第二层网络结构
        conv2_weights = tf.Variable("weight",[CONV2_SIZE,CONV2_SIZE,CONV1_DEEP,CONV2_DEEP],
                                   initializer=tf.truncated_normal_initializer(stddev=0.1)),
        conv2_biases = tf.get_variable("bias",[CONV2_DEEP],initializer=tf.constant_initializer(0.0))
        conv2 = tf.nn.conv2d(pool1,conv2_weights,strides=[1,1,1,1],padding="VALID")
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2,conv2_biases))
    with tf.name_scope("pool2"):
        pool2 = tf.nn.max_pool(relu2,ksize=[1,2,2,1],stides=[1,2,2,1],padding='VALID')
    pool_shape = pool2.get_shape().as_list()
    nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]
    reshaped = tf.reshape(pool2,[pool_shape[0],nodes])
    with tf.variable_scope("fc1"):
        fc1_weights = tf.get_variable("weight",[nodes,FC_SIZE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if(regularizer != None):
             tf.add_to_collection('losses',regularizer(fc1_weights))
        fc1_biases = tf.get_variable("bias",[FC_SIZE],initializer=tf.constant_initializer(0.1))
        fc1 = tf.nn.relu(tf.matmul(reshaped,fc1_weights) + fc1_biases)
#         if(is_train):
#             fc1 = tf.nn.dropout(fc1,0.5) #0.5的概率将这个点的值变成0
    with tf.variable_scope("fc2"):
        fc2_weights = tf.get_variable("weight",[FC_SIZE,FC2_SIZE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if(regularizer != None):
            tf.add_to_collection('losses',regularizer(fc2_weights))
        fc2_bRATEiases = tf.get_variable("bias",[FC2_SIZE],initializer=tf.constant_initializer(0.1))
        fc2 = tf.nn.relu(tf.matmul(fc1,fc2_weights) + fc2_biases)
#         if(is_train):
#             fc2 = tf.nn.dropout(fc2,0.5)
    with tf.variable_scope("fc3"):
        fc3_weights = tf.get_variable("weight",[FC2_SIZE,FC3_SIZE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if(regularizer != None):
            tf.add_to_collection('losses',regularizer(fc3_weights))
        fc3_biases = tf.get_variable("bias",[FC3_SIZE],initializer=tf.constant_initializer(0.1))
        fc3 = tf.nn.relu(tf.matmul(fc2,fc3_weights) + fc3_biases)

In [78]:
BATCH_SIZE = 175
LEARNING_BASE_RATE = 0.8   #基础的学习率
LEARNING_RATE_DECAY = 0.99 #学习率的衰减率
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99

MODEL_SAVE_PATH = "./model"
MODEL_NAME = "model.ckpt"

In [84]:
def def back_propagation():
    x_train = tf.placeholder(tf.float32,[None,INPUT_NODE],name="x-input")
    y_train = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='y-input')
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y_predict = forward_propagation(x,regularizer)
    
    training_times = tf.Variable(0,trainable=False)
    #滑动平均
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    #损失函数
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = y_predict,label=tf.argmax(y_train,1))#tf.argmax(y_train,1) 将one_hot编码格式下的数据转换为对应的label
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection("losses"))
    #学习率
    learning_rate = tf.train.exponential_decay(LEARNING_BASE_RATE,training_times,len(data_train)/BATCH_SIZE,LEARNING_RATE_DECAY)
    #梯度下降法优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,training_times)
    with tf.control_dependencies([train_step,variables_averages_op]):
        train_op = tf.no_op(name = 'train')
    saver = tf.train.Saver()
    with tf

In [ ]:
def train():
    

## 对需要预测的数据 

In [ ]:
df_test = pd.read_csv("./sample_submission.csv")

## 为了测试 

In [ ]:
a = tf.constant([[1.0,2.0],[1.,2.]],name="a")
b = tf.constant([2.0,3.0],name="b")
result = tf.add(a,b,name="add3")https
print (result)

In [63]:
from tensorflow.examples.tutorials.mnist import input_data

In [64]:
type(input_data)

module

In [71]:
mnist = input_data.read_data_sets("/temp/data",one_hot=True)
mnist.train.next_batch(1)

Extracting /temp/data/train-images-idx3-ubyte.gz
Extracting /temp/data/train-labels-idx1-ubyte.gz
Extracting /temp/data/t10k-images-idx3-ubyte.gz
Extracting /temp/data/t10k-labels-idx1-ubyte.gz


(array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0

In [90]:
data_train

['./processedPictures/5433_1.jpg',
 './processedPictures/12066_1.jpg',
 './processedPictures/16971_1.jpg',
 './processedPictures/1397_1.jpg',
 './processedPictures/280_0.jpg',
 './processedPictures/17068_1.jpg',
 './processedPictures/749_1.jpg',
 './processedPictures/10100_0.jpg',
 './processedPictures/15622_1.jpg',
 './processedPictures/1859_1.jpg',
 './processedPictures/5536_0.jpg',
 './processedPictures/14350_1.jpg',
 './processedPictures/16185_1.jpg',
 './processedPictures/17357_0.jpg',
 './processedPictures/14466_0.jpg',
 './processedPictures/13486_1.jpg',
 './processedPictures/10815_1.jpg',
 './processedPictures/9318_0.jpg',
 './processedPictures/5147_1.jpg',
 './processedPictures/8475_1.jpg',
 './processedPictures/5799_1.jpg',
 './processedPictures/13502_0.jpg',
 './processedPictures/6145_1.jpg',
 './processedPictures/17373_1.jpg',
 './processedPictures/680_0.jpg',
 './processedPictures/4536_0.jpg',
 './processedPictures/1764_1.jpg',
 './processedPictures/5409_1.jpg',
 './proces

In [91]:
label_train

['1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1'

In [82]:
tf.argmax([0,0,0,1,0],1)


<tf.Tensor 'ArgMax_2:0' shape=() dtype=int64>